In [1]:
import glob, json, os
import numpy as np

In [2]:
def readJson(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

# (q0,q1,q2,q3) = (W, X, Y, Z)
def quaternion_to_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

def convertTransformMatrix(translation, quaternion):
    
    transform = np.identity(4)

    # T: (X, Y, Z)
    translation = np.array(translation)
    transform[0:3, 3] = translation
    
    # Q: (W, X, Y, Z)
    rotation = quaternion_to_rotation_matrix(quaternion)
    transform[0:3, 0:3] = rotation

    return transform

In [3]:
# Parse 49195 complex brdf objects
brdf_paths = glob.glob('/home/leyinghu/Documents/data/complex_brdf/specular/*/metadata.json')
brdf_paths.sort()
assert len(brdf_paths) == 49195

In [4]:
for brdf_path in brdf_paths:
    
    brdf_dir = os.path.dirname(brdf_path)
    pose_dir = os.path.join(brdf_dir, 'pose')

    if not os.path.isdir(pose_dir):
        os.mkdir(pose_dir)
        
    brdf_data = readJson(brdf_path)
    brdf_images = glob.glob(os.path.join(brdf_dir, 'rgb', '*.png'))
    assert brdf_data['metadata']['num_frames'] == len(brdf_images)

    translations = brdf_data['camera']['positions']
    quaternions = brdf_data['camera']['quaternions']
    
    for i in range(len(translations)):
        
        pose = convertTransformMatrix(translations[i], quaternions[i])
        pose_path = os.path.join(pose_dir, f"{i:06d}.txt")
        np.savetxt(pose_path, pose)